In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import torch

/home/leo/NLP/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_dir = "/home/leo/NLP/models/Qwen-VL/Qwen2.5-3B"

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True, padding_side='left')

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="cuda",
    quantization_config=quantization_config,
)
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


In [3]:
targets = ["q_proj", "k_proj", "v_proj", "o_proj", "lm_head"]
target_layers = list(range(len(model.model.layers)))[-5:]

lora_conf = LoraConfig(
        r=8,
        target_modules=targets,
        lora_dropout=0.1,
        bias='none',
        layers_to_transform=target_layers
)

lora_model = get_peft_model(model, lora_conf)

In [4]:
parameters_to_train = filter(lambda p: p.requires_grad, lora_model.parameters())
optimizer = torch.optim.AdamW(parameters_to_train, lr = 3e-4)

In [5]:
def set_grad_hook_false(module, args) -> None:
    torch.set_grad_enabled(False)
    return None

def set_grad_hook_true(module, args) -> None:
    torch.set_grad_enabled(True)
    return None

model.model.embed_tokens.register_forward_pre_hook(set_grad_hook_false)
model.model.layers[target_layers[0]].register_forward_pre_hook(set_grad_hook_true)

In [6]:
test_seq = [
    "除了庞大的数据集和强大的硬件， 优秀的软件工具在深度学习的快速发展中发挥了不可或缺的作用。 从2007年发布的开创性的Theano库开始， 灵活的开源工具使研究人员能够快速开发模型原型， 避免了我们使用标准组件时的重复工作， 同时仍然保持了我们进行底层修改的能力。 随着时间的推移，深度学习库已经演变成提供越来越粗糙的抽象。 就像半导体设计师从指定晶体管到逻辑电路再到编写代码一样， 神经网络研究人员已经从考虑单个人工神经元的行为转变为从层的角度构思网络， 通常在设计架构时考虑的是更粗糙的块（block）。之前我们已经介绍了一些基本的机器学习概念， 并慢慢介绍了功能齐全的深度学习模型。 在上一章中，我们从零开始实现了多层感知机的每个组件， 然后展示了如何利用高级API轻松地实现相同的模型。 为了易于学习，我们调用了深度学习库，但是跳过了它们工作的细节。 在本章中，我们将深入探索深度学习计算的关键组件， 即模型构建、参数访问与初始化、设计自定义层和块、将模型读写到磁盘， 以及利用GPU实现显著的加速。 ",
    """之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。 在这里，整个模型只有一个输出。 注意，单个神经网络 （1）接受一些输入； （2）生成相应的标量输出； （3）具有一组相关 参数（parameters），更新这些参数可以优化某目标函数。然后，当考虑具有多个输出的网络时， 我们利用矢量化算法来描述整层神经元。 像单个神经元一样，层（1）接受一组输入， （2）生成相应的输出， （3）由一组可调整参数描述。 当我们使用softmax回归时，一个单层本身就是模型。 然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。对于多层感知机而言，整个模型及其组成层都是这种架构。 整个模型接受原始输入（特征），生成输出（预测）， 并包含一些参数（所有组成层的参数集合）。 同样，每个单独的层接收输入（由前一层提供）， 生成输出（到下一层的输入），并且具有一组可调参数， 这些参数根据从下一层反向传播的信号进行更新。""",
    """事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。 例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层， 这些层是由层组（groups of layers）的重复模式组成。 这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛 的识别和检测任务 (He et al., 2016)。 目前ResNet架构仍然是许多视觉任务的首选架构。 在其他的领域，如自然语言处理和语音， 层组以各种重复模式排列的类似架构现在也是普遍存在。为了实现这些复杂的网络，我们引入了神经网络块的概念。 块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 使用块进行抽象的一个好处是可以将一些块组合成更大的组件， 这一过程通常是递归的，如 图5.1.1所示。 通过定义代码来按需生成任意复杂度的块， 我们可以通过简洁的代码实现复杂的神经网络。"""
    ]

encoded_seq = tokenizer(test_seq, return_tensors="pt", padding=True, add_special_tokens=False)
encoded_seq["labels"] = encoded_seq["input_ids"]

encoded_seq['input_ids'].shape

torch.cuda.memory._record_memory_history()
out = lora_model(**encoded_seq)
torch.cuda.memory._dump_snapshot("/dev/shm/my_snapshot.pickle")
# https://pytorch.org/memory_viz

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

In [7]:
out.loss.backward()
optimizer.step()
optimizer.zero_grad()